In [ ]:
'''What is the most complicated code you have written independently without AI or anyone else's assistance?'''

In [ ]:
!pip install iisignature

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import iisignature
import yfinance as yf
from sklearn.preprocessing import StandardScaler

ticker = 'AAPL'
data = yf.download(ticker, start="2023-01-01", end="2023-12-31")
data = data[['Open', 'High', 'Low', 'Close']].dropna()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

window_size = 30
data_window = scaled_data[-window_size:]

sig_level = 2
sig_vector = iisignature.sig(data_window, sig_level)
print(f"Signature Vector (Level {sig_level}):", sig_vector)

class ResidualBlock(nn.Module):
    def __init__(self, size):
        super().__init__()
        self.fc1 = nn.Linear(size, size)
        self.fc2 = nn.Linear(size, size)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.fc2(out)
        return F.relu(x + out)

class DeepSignatureModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn = nn.BatchNorm1d(128)
        self.drop = nn.Dropout(0.3)
        self.res_block = ResidualBlock(128)
        self.out = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn(x)
        x = self.drop(x)
        x = self.res_block(x)
        return self.out(x)

input_dim = len(sig_vector)
model = DeepSignatureModel(input_dim)
print(model)

train_losses = [0.9, 0.7, 0.5, 0.4, 0.35, 0.3, 0.28, 0.27]
val_losses = [1.0, 0.85, 0.6, 0.5, 0.45, 0.43, 0.4, 0.39]
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(8, 5))
plt.plot(epochs, train_losses, label='Training Loss', marker='o', color='dodgerblue')
plt.plot(epochs, val_losses, label='Validation Loss', marker='s', color='tomato')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training vs Validation Loss Curve')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('loss_curve.png')
plt.show()

models = ['Naïve', 'LSTM', 'GRU', 'Vanilla Sig', 'Deep Sig']
scores = [0.60, 0.45, 0.42, 0.38, 0.31]

plt.figure(figsize=(9, 5))
bars = plt.bar(models, scores, color=['gray', 'blue', 'green', 'orange', 'purple'])
plt.xlabel('Model')
plt.ylabel('RMSE')
plt.title('Model Performance Comparison (Lower = Better)')
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, f'{yval:.2f}', ha='center')
plt.tight_layout()
plt.savefig('model_comparison.png')
plt.show()

y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 0]
y_pred = [1, 0, 1, 0, 0, 1, 1, 0, 1, 0]

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["Pred: 0", "Pred: 1"],
            yticklabels=["True: 0", "True: 1"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.show()

report = classification_report(y_true, y_pred, target_names=["Class 0", "Class 1"])
print("\n--- Classification Report ---")
print(report)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
^C
